In [ ]:
# Logistics:

In [ ]:
%pip install parliamentarch ipywidgets

In [ ]:
import parliamentarch
import ipywidgets as widgets

In [ ]:
from IPython.display import SVG, display
def show_svg(svg_string):
    display(SVG(svg_string))

In [ ]:
import hashlib

def color_hash(string: str):
    return "#" + hashlib.sha256(string.encode('utf-8')).hexdigest()[:6].upper()

In [ ]:
class DefaultColorDict(dict):
    def __missing__(self, key):
        value = color_hash(key)
        # insert value into dict and return value
        self[key] = value
        return value

In [ ]:
party_colors = DefaultColorDict(
{
    'PvdA-GL': 'red',
    'D66': 'green',
    'CDA': '#444',
    'VVD': 'blue',
    'SGP': 'orange',
    'DENK': 'cyan',
    'Volt': 'purple',
    'rest': '#ccc',
})

In [ ]:
def display_seats(seats: dict[str, int], arch_canvas_size=175, arch_margins=5):
    attrib = {parliamentarch.SeatData(name, party_colors[name]): number for name, number in seats.items()}
    svg = parliamentarch.get_svg_from_attribution(attrib, canvas_size=arch_canvas_size, margins=arch_margins)
    show_svg(svg)

In [ ]:
# Main app:

In [ ]:
# tweede kamer vanaf november 2025
tweede_kamer_seats = {
    "D66": 26,
    "PVV": 26,
    "VVD": 22,
    "PvdA-GL": 20,
    "CDA": 18,
    "JA21": 9,
    "FvD": 7,
    "BBB": 4,
    "PvdD": 3,
    "DENK": 3,
    "SGP": 3,
    "CU": 3,
    "SP": 3,
    "50PLUS": 2,
    "Volt": 1,
}

In [ ]:
# huidige eerste kamer
eerste_kamer_seats = {
    "BBB": 13,
    "VVD": 9,
    "PVV": 4,
    "PvdA-GL": 14,
    "CDA": 6,
    "D66": 6,
    "SP": 3,
    "PvdD": 3,
    "JA21": 2,
    "CU": 3,
    "SGP": 2,
    "Volt": 2,
    "FvD": 3,
    "50PLUS": 1,
    "OSF/OPNL": 1,
    "Walenkamp (ex-BBB)": 1,
    "van de Sanden (ex-VVD)": 1,
    "Beukering (ex-JA21)": 1,
}

In [ ]:
partijen = sorted(set(tweede_kamer_seats.keys()) | set(eerste_kamer_seats.keys()), key=lambda p: (-tweede_kamer_seats.get(p, 0) - eerste_kamer_seats.get(p, 0), p))

In [ ]:
eerste_kamer_selectie = {"rest": 75}
tweede_kamer_selectie = {"rest": 150}

# layout
arch_canvas_size = 175 # px height, half the width
arch_margins = 5 # px

output_layout = {
    # 'width': '100%',
    'width': f'{arch_margins * 2 + arch_canvas_size * 2}px',
}

output = widgets.Output(layout=output_layout)

def display_archs():
    with output:
        tweede_kamer_coalitie_seats = sum(tweede_kamer_selectie.values()) - tweede_kamer_selectie["rest"]
        if tweede_kamer_coalitie_seats > sum(tweede_kamer_selectie.values()) / 2:
            tweede_kamer_coalitie_status = " (meerderheid)"
        else:
            tweede_kamer_coalitie_status = ""
        print(f"Tweede Kamer: {tweede_kamer_coalitie_seats} zetels{tweede_kamer_coalitie_status}")
        display_seats(tweede_kamer_selectie, arch_canvas_size=arch_canvas_size, arch_margins=arch_margins)

        eerste_kamer_coalitie_seats = sum(eerste_kamer_selectie.values()) - eerste_kamer_selectie["rest"]
        if eerste_kamer_coalitie_seats > sum(eerste_kamer_selectie.values()) / 2:
            eerste_kamer_coalitie_status = " (meerderheid)"
        else:
            eerste_kamer_coalitie_status = ""
        print(f"Eerste Kamer: {eerste_kamer_coalitie_seats} zetels{eerste_kamer_coalitie_status}")
        display_seats(eerste_kamer_selectie, arch_canvas_size=arch_canvas_size, arch_margins=arch_margins)

def on_change(change):
    output.outputs = []
    output.clear_output() # for vscode notebook support; not needed in Voila/Jupyter/Voici
    owner = change["owner"]

    rest_2K_oud = tweede_kamer_selectie.pop('rest')
    rest_1K_oud = eerste_kamer_selectie.pop('rest')
    if change["new"]: # checkbox checked
        tweede_kamer_selectie[owner.party_name] = owner.seats2K
        tweede_kamer_selectie['rest'] = rest_2K_oud - owner.seats2K
        eerste_kamer_selectie[owner.party_name] = owner.seats1K
        eerste_kamer_selectie['rest'] = rest_1K_oud - owner.seats1K
        # color the checkbox label with party color
        owner.style.background = party_colors[owner.party_name]
    else:             # checkbox unchecked
        tweede_kamer_selectie.pop(owner.party_name)
        tweede_kamer_selectie['rest'] = rest_2K_oud + owner.seats2K
        eerste_kamer_selectie.pop(owner.party_name)
        eerste_kamer_selectie['rest'] = rest_1K_oud + owner.seats1K
        # reset color
        owner.style.background = "transparent"

    display_archs()

checkboxes = []

for party_name in partijen:
    party_seats_2K = tweede_kamer_seats.get(party_name, 0)
    party_seats_1K = eerste_kamer_seats.get(party_name, 0)
    widget = widgets.Checkbox(
        value=False,
        description=f"{party_name} (zetels: {party_seats_2K} 2e Kamer, {party_seats_1K} 1e Kamer)",
        disabled=False,
        indent=False
    )
    widget.party_name = party_name
    widget.seats2K = party_seats_2K
    widget.seats1K = party_seats_1K
    widget.observe(on_change, names='value')
    checkboxes.append(widget)

display(widgets.HTML(value="<b>Selecteer partijen om hun zetels weer te geven in beide kamers.</b>"))
checkboxes_vbox = widgets.VBox(checkboxes, disabled=False)
box = widgets.Box([output, checkboxes_vbox], disabled=False)

display_archs()

display(box)